In [15]:
import numpy as np
import os
import pandas as pd
import xarray as xr
from pathlib import Path
from utils.data_reading.sound_data.station import StationsCatalog

In [16]:
catalog_path = "/media/imonge/CORSAIR"
ERA5_dir = "../../../../data/test_correlation/ERA5_concat"
output_dir = "../../../../data/test_correlation/data_soundscape/combined_wave_index"
stations = StationsCatalog(catalog_path)

In [17]:
for station in stations:
    hydro_number = station.name[-1]
    station_code = f"h{hydro_number}"
    out_path = os.path.join(output_dir, f"{station_code}.pkl")
    if Path(out_path).exists():
        continue

    date_start = station.date_start
    date_end = station.date_end

    ERA5_input_path = os.path.join(ERA5_dir, f"ERA5_MAHY*{hydro_number}_waves_merged.nc")
    data = xr.open_dataset(ERA5_input_path)

    t = pd.to_datetime(data["valid_time"].values)

    swh = np.array(data["swh"]).flatten()
    mwp = np.array(data["mwp"]).flatten()

    combined_wave_index = swh * mwp

    df = pd.DataFrame({'values': combined_wave_index}, index=t)

    df.to_pickle(out_path)